In [4]:
#Opencv Python - Similarity score from Feature matching + Homograpy
#https://answers.opencv.org/question/60350/opencv-python-similarity-score-from-feature-matching-homograpy/
#Homography : http://www.gisdeveloper.co.kr/?p=6832
#입력 데이터를 비슷한 크기로 가져야 한다, 아니라면 너무 많은 특징이 검출될 수 있다

import cv2
import numpy as np
import os

In [5]:
def feature_matcher(query_image, image_folder):

    min_match_count = 10

    img1 = cv2.imread(query_image, 0)
    surf = cv2.xfeatures2d.SURF_create(800)
    kp1, des1 = surf.detectAndCompute(img1, None)

    bf = cv2.BFMatcher(cv2.NORM_L2)

    path_files = os.getcwd()
    all_files = next(os.walk('.'+image_folder))[2]
#    print(path_files + all_files[0])

    good_matches = []
    match_max = -1
    match_img = ''
    
    cnt = 0
    
    for file_name_temp in all_files:
        cnt += 1;
        print(cnt/len(all_files)*100.0,'%')
        try:
            train_image = path_files + image_folder + '/' + file_name_temp
#            print(train_image)
            
#             t = cv2.imread(train_image, cv2.IMREAD_COLOR)
#             cv2.imshow('img1', img1)
#             cv2.imshow('test', t)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
#             cv2.waitKey(1)
            
            dst = cv2.imread(train_image, 0)
            img2 = cv2.resize(dst, dsize=(img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_AREA)
        
            surf = cv2.xfeatures2d.SURF_create(800)
            kp2, des2 = surf.detectAndCompute(img2, None)

#             print(1)
            matches = bf.knnMatch(des1, des2, k=2)

#             print(2)
            good = []
            for m, n in matches:
                if m.distance < 0.7*n.distance:
                    good.append(m)
                    
#             print(3)
            if len(good) > min_match_count:

                src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
                dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
                
#                 print(4)
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

#                 print(5)
                matchesMask = mask.ravel().tolist()

#                 print(6)
                h, w = img1.shape
                pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts, M)

#                 print(7)
#                 if not M==None:
#                     print(8)
#                     print()
#                     print("-" * 2, file_name_temp)
#                     print("number of good matches", len(good))
#                     print("*" * 10, matchesMask)

                #Get matches avg, max, min
                good_matches.append(len(good))
    
#                 print("-" * 2, file_name_temp)
#                 print("number of good matches", len(good))
                
                #Get max matches img, len(good)
                if match_max < len(good):
                    match_max = len(good)
                    match_img = file_name_temp
                
#                 print("*" * 10, matchesMask)              
#                 print(type(matchesMask))
#                 np_matchedMask = np.array(matchesMask, dtype=np.bool)
#                 print(type(np_matchedMask))
#                 print(np.count_nonzero(np_matchedMask), len(np_matchedMask))
                print()
#                 print(9)
            
        except Exception as ex: # 에러 종류
            print('에러가 발생 했습니다', ex)
            
    print(good_matches)
    print(max(good_matches))
    print(min(good_matches))
#     print(match_max)
    print(match_img)
#     print(good_matches.index(max(good_matches)))
#     print(all_files[good_matches.index(max(good_matches))+1])
    print(1.0 * sum(good_matches) / len(good_matches))
    featureMatching(query_image, path_files + image_folder + '/' + match_img)
                    

In [6]:
def featureMatching(img_url1, img_url2):
    img1 = cv2.imread(img_url1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img_url2, cv2.IMREAD_GRAYSCALE)
    res = None
    
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    
    matches = sorted(matches, key=lambda x:x.distance)
    res = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], res, flags=0)
    
    cv2.imshow('Feature Matching', res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [7]:
# query_image = '/home/zealous/Pictures/train_images/AbudefdufWhitleyiJER.jpg'
# trained_image_folder = '/home/zealous/Pictures/train_images'

#query_image = './image/test/img0.png'
# query_image = './image/test/img8.png'
query_image = './image/test/t_img_1.jpeg'
image_folder = '/image/train'
feature_matcher(query_image, image_folder)

0.32786885245901637 %

0.6557377049180327 %

0.9836065573770493 %

1.3114754098360655 %

1.639344262295082 %

1.9672131147540985 %

2.2950819672131146 %

2.622950819672131 %

2.9508196721311477 %

3.278688524590164 %

3.606557377049181 %

3.934426229508197 %

4.2622950819672125 %

4.590163934426229 %

4.918032786885246 %

5.245901639344262 %

5.573770491803279 %

5.901639344262295 %

6.229508196721312 %

6.557377049180328 %

6.885245901639345 %

7.213114754098362 %

7.540983606557377 %

7.868852459016394 %

8.19672131147541 %

8.524590163934425 %

8.852459016393443 %

9.180327868852459 %

9.508196721311474 %

9.836065573770492 %

10.163934426229508 %

10.491803278688524 %

10.819672131147541 %

11.147540983606557 %

11.475409836065573 %

11.80327868852459 %

12.131147540983607 %

12.459016393442624 %

12.786885245901638 %

13.114754098360656 %

13.442622950819672 %

13.77049180327869 %

14.098360655737704 %

14.426229508196723 %

14.754098360655737 %

15.081967213114755 %

15.409836065

KeyboardInterrupt: 